In [2]:
import pandas as pd
matches = pd.read_csv('/home/nanthony007/VeritasAnalytics-Website/collection/stats/matchdata.csv')
events = pd.read_csv('/home/nanthony007/VeritasAnalytics-Website/collection/stats/events.csv')
ts = pd.read_csv('/home/nanthony007/VeritasAnalytics-Website/collection/stats/timeseries.csv')
wrestlers = pd.read_csv('/home/nanthony007/VeritasAnalytics-Website/collection/stats/wrestlers.csv')
wrestlers.head()

,name,rating,team_id
0,Robert Rehm,968,USA
1,Paul Glynn,984,USA
2,Blaize Cabell,959,USA
3,Ben Whitford,1038,USA
4,Johnni DiJulius,993,USA


In [3]:
# calculates effective wins and assigns row-wise
for person in matches.focus_id.unique():
    group = matches[matches['focus_id']==person]
    ew = group.num_result.mean() * len(group.index)
    for i, row in wrestlers.iterrows():
        if row['name'] == person:
            wrestlers.at[i, 'ew'] = round(ew, 2)

In [4]:
wrestlers.sort_values('ew', ascending=False).head()

,name,rating,team_id,ew
332,Jason Nolf,1130,USA,15.45
312,Nathan Jackson,1108,USA,14.75
109,Tyler Graff,1160,USA,14.65
98,Garrett Ryan,1105,USA,14.50
213,Jaydin Eierman,1105,USA,14.05


In [5]:
!pwd

/home/nanthony007/VeritasAnalytics-Website/collection/analysis
